# ランダムフォレスト

付近のwifiアクセスポイントをランダムフォレスト(機会学習)で学習し、登録されたどの位置に近いかを判定する

required sklearn, access-points

```
$ pip install access-points sklearn
```

もしくは

```
$ pip install -r requirements.txt
```

In [ ]:
import os, pickle, io, csv, json

from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction import DictVectorizer
from access_points import get_scanner

### アクセスポイント(データセット)の取得関数

In [ ]:
def scan_access_points(label='default place', size=1):
    print('start scanning {} access points {} times.'.format(label, size))
    X = []
    y = []
    wifi_scanner = get_scanner()
    for index in range(int(size)):
        print('scan ' + str(index))
        access_points = wifi_scanner.get_access_points()
        X.append({' '.join([ap.ssid, ap.bssid]): ap.quality for ap in access_points })
        y.append(label)
    return X, y

データセットの中身

In [ ]:
X, y = scan_access_points()
for Xi, yi in zip(X, y):
    print('{} access points = {}'.format(yi, Xi))

### モデルの定義

ランダムフォレストを定義。
DictVectorizerで、hash型のデータをいい感じに補完してもらう(ないkeyは0にするなど)

In [ ]:
model = make_pipeline(
    DictVectorizer(sparse=False),
    RandomForestClassifier(n_estimators=500, class_weight='balanced')
)

### サンプリング(データ集め)

In [ ]:
X = []
y = []

リビングに移動して下記を実行します。もしくはlabelを書き換えてください。

In [ ]:
_X, _y = scan_access_points(label='living room', size=10)
X.extend(_X)
y.extend(_y)

寝室に移動して下記を実行します。もしくはlabelを書き換えてください。

In [ ]:
_X, _y = scan_access_points(label='bedroom room', size=10)
X.extend(_X)
y.extend(_y)

トイレに移動して下記を実行します。もしくはlabelを書き換えてください。

In [ ]:
_X, _y = scan_access_points(label='toilet', size=1)
X.extend(_X)
y.extend(_y)

### トレーニング実行

In [ ]:
model.fit(X, y)

### 登録されている場所

In [ ]:
place_names = model.classes_
print(place_names)

### 場所の推論

登録されている場所に移動して実行します

In [ ]:
X, _ = scan_access_points(size=1)
place_name = model.predict(X[0])[0]
print(place_name)

### 推論結果の割合

登録されている場所に移動して実行します

In [ ]:
X, _ = scan_access_points(size=1)
percentages = {x: y for x, y in zip(place_names, model.predict_proba(X[0])[0])}
print(percentages)